<a href="https://colab.research.google.com/github/sancia15/Corona-Bot/blob/master/coronachatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

#link of dataset
pd.set_option('max_colwidth', 100)  # Increase column width
data = pd.read_excel("Copy of WHO_FAQ.xlsx", encoding='utf8')#datset link
data.head()

,Context,Answer
0,What is a coronavirus?,Coronaviruses are a large family of viruses which may cause illness in animals or humans.
1,What is a coronavirus?,"In humans, several coronaviruses are known to cause respiratory infections ranging from the comm..."
2,What is COVID-19?,COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new ...
3,What are the symptoms of COVID-19?,"The most common symptoms of COVID-19 are fever, tiredness, and dry cough. Some patients may have..."
4,What are the symptoms of COVID-19?,Some people become infected but don’t develop any symptoms and don't feel unwell. Most people (a...


In [0]:

!pip install -q tensorflow_text

     |████████████████████████████████| 3.0MB 2.8MB/s 


In [0]:
#USE pretrained model to extract response encodings.
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import re
def preprocess_sentences(input_sentences):
    return [re.sub(r'(covid-19|covid)', 'coronavirus', input_sentence, flags=re.I) 
            for input_sentence in input_sentences]
        
# Load module containing USE
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

# Create response embeddings
response_encodings = module.signatures['response_encoder'](
        input=tf.constant(preprocess_sentences(data.Answer)),
        context=tf.constant(preprocess_sentences(data.Context)))['outputs']



In [0]:
test_questions = [
    "What is corona virus?",
    "What are the symptoms of COVID-19?",
    "Are animals contagious COVID-19?",
    "Are there medicine against the coronavirus?",
    "Is there a vaccine, drug or treatment for COVID-19?",
    "Is COVID-19 the same as SARS?",
    "Can CoVID-19 be caught from a person who has no symptoms?",
    "Should I stay inside the house?",
    "Kann ich mit meinem Hund spazieren gehen?"  #Other languages are also valid
]

# encodings for test questions
question_encodings = module.signatures['question_encoder'](
    tf.constant(preprocess_sentences(test_questions))
)['outputs']

# Get the responses
test_responses = data.Answer[np.argmax(np.inner(question_encodings, response_encodings), axis=1)]

# Show them in a dataframe
pd.DataFrame({'Test Questions': test_questions, 'Test Responses': test_responses})

,Test Questions,Test Responses
45,What is corona virus?,"Coronaviruses are a large family of viruses that are common in animals. Occasionally, people get..."
11,What are the symptoms of COVID-19?,Many people with COVID-19 experience only mild symptoms. This is particularly true at the early ...
45,Are animals contagious COVID-19?,"Coronaviruses are a large family of viruses that are common in animals. Occasionally, people get..."
32,Are there medicine against the coronavirus?,"Not yet. To date, there is no vaccine and no specific antiviral medicine to prevent or treat COV..."
32,"Is there a vaccine, drug or treatment for COVID-19?","Not yet. To date, there is no vaccine and no specific antiviral medicine to prevent or treat COV..."
37,Is COVID-19 the same as SARS?,SARS was more deadly but much less infectious than COVID-19. There have been no outbreaks of SAR...
11,Can CoVID-19 be caught from a person who has no symptoms?,Many people with COVID-19 experience only mild symptoms. This is particularly true at the early ...
17,Should I stay inside the house?,"Stay home if you feel unwell. If you have a fever, cough and difficulty breathing, seek medical ..."
18,Kann ich mit meinem Hund spazieren gehen?,"If possible, avoid traveling to places – especially if you are an older person or have diabetes..."
